In [1]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import time

## WORDNET 이용한 유의어 추출하기

In [2]:
from nltk.corpus import wordnet as wn

search_keywords_list = ['you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife']

sim_words_list = []
for conf_word in search_keywords_list:
    for synset in wn.synsets(conf_word):
        sim_words_list.append(synset.name()) #동의, 유의어
        #sim_words_list.append(synset.hypernyms()) # 상위어
        #sim_words_list.append(synset.hyponyms()) # 하위어
        
conf_words_flttend_list = ''.join(str(sim_words_list)) #문자열로 변환
result = re.findall(r"(?i)\b[a-z]+\b",conf_words_flttend_list)
result_ =[]
for i in result:
    if i != 'n' and i != 'v' and i != 's' and i != 'a':
        result_.append(i)
        
search_keywords_list_re = set(result_)
search_keywords_list_re 

{'aunt',
 'baby',
 'beget',
 'brother',
 'buddy',
 'conserve',
 'counterpart',
 'cousin',
 'daughter',
 'duplicate',
 'ex',
 'gemini',
 'grandchild',
 'granddaughter',
 'grandfather',
 'grandma',
 'he',
 'helium',
 'husband',
 'in',
 'law',
 'match',
 'mother',
 'nephew',
 'niece',
 'parent',
 'person',
 'rear',
 'sister',
 'son',
 'stepdaughter',
 'stepfather',
 'stepmother',
 'stepson',
 'twin',
 'uncle',
 'widow',
 'widower',
 'wife'}

In [3]:
def character_all_section(text):


    # Number of Characters
    def NumberofCharacters(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
#         for i in filtered_chr_text__:
#             ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count



    # number_of_characters = NumberofCharacters(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # number_of_characters
    # print ('=============================================')
    # print ('Number of Characters :', number_of_characters)
    # print ('=============================================')

    ####################################
    #### Character Descriptiveness #####
    ####################################

    def character_descrip(text):

        input_sentence = text

        def findSentence(input_sentence):
            result = []

            data = str(input_sentence)
            #data = input_sentence.splitlines()
            
            findText = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']

            sentences = data.split(".")
            
            for sentence in sentences:
                for item in findText:
                    if item in sentence:
                        result.append(sentence)

            return result

        input_sent_included_character = findSentence(text) 
        input_sent_chr = set(input_sent_included_character) #중복값을 제거해보자
        input_sent_chr = '.'.join(input_sent_chr) #하나의 문자열로 합쳐야 원본 문장처럼 변환되고, 이것을 show/tell 분석코드에 넣게됨



        #입력된 전체 문장을 개별문장으로 분리하여 전처리 처리함
        def sentence_to_df(input_sentence):

            input_text_df = nltk.tokenize.sent_tokenize(input_sentence)
            test = []

            for i in range(0,len(input_text_df)):
                new_label = np.random.randint(0,2)  # 개별문장(input_text_df) 수만큼 0 또는 1 난수 생성
                data = [new_label, input_text_df[i]]
                test.append(data)

            #print(test)
            dataf = pd.DataFrame(test, columns=['label', 'text'])
            #print(dataf)
            return dataf


        class STDataset(Dataset):
            ''' Showing Telling Corpus Dataset '''
            def __init__(self, df):
                self.df = df

            def __len__(self):
                return len(self.df)

            def __getitem__(self, idx):
                text = self.df.iloc[idx, 1]
                label = int(self.df.iloc[idx, 0])
                return text, label


        ###########입력받은 데이터 처리 실행하는 메소드 showtell_classfy() ###############
        #result_all.html에서 입력받을 text를 contents에 넣고 전처리 후 데이터프레임에 넣어줌
        def showtell_classfy(text):
            contents = str(text)
            preprossed_contents_df = sentence_to_df(contents)

            preprossed_contents_df.dropna(inplace=True)
            #전처리된 데이터를 확인(데이터프레임으로 가공됨)
            preprossed_contents_df__ = preprossed_contents_df.sample(frac=1, random_state=999)
            

            #파이토치에 입력하기 위해서 로딩...
            ST_test_dataset = STDataset(preprossed_contents_df__)
            test_loader = DataLoader(ST_test_dataset, batch_size=1, shuffle=True, num_workers=0)
            #로딩되는지 확인
            ST_test_dataset.__getitem__(1)

            #time.sleep(1)



            #check whether cuda is available
            #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
            device = torch.device("cpu")  
            #device = torch.device("cuda")
            #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
            tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
            model = BertForSequenceClassification.from_pretrained('bert-large-cased')
            model.to(device)



            # for text, label in test_loader :
            #     print("text:",text)
            #     print("label:",label)


            #저장된 모델을 불러온다.
            #J:\Django\EssayFit_Django\essayfitaiproject\essayfitapp\model.pt
            #time.sleep(1)
            #model = torch.load("/Users/jongphilkim/Desktop/Django_WEB/essayfitaiproject/essayai/model.pt", map_location=torch.device('cpu'))
            model = torch.load("model.pt", map_location=torch.device('cpu'))
            print("model loadling~")
            model.eval()


            pred_loader = test_loader
            print("pred_loader:", pred_loader)
            total_loss = 0
            total_len = 0
            total_showing__ = 0
            total_telling__ = 0

            showing_conunter = [] #문장에 해당하는 SHOWING을 계산한다.
            
            print("check!")
            for text, label in pred_loader:
                print("text:",text)
                #print("label:",label)
                encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] #text to tokenize
                padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] #padding
                sample = torch.tensor(padded_list) #torch tensor로 변환
                sample, label = sample.to(device), label.to(device) #tokenized text에 label을 넣어서 Device(gpu/cpu)에 넣기 위해 준비
                labels = torch.tensor(label) #레이블을 텐서로 변환
                #time.sleep(1)
                outputs = model(sample,labels=labels) #모델을 통해서 샘플텍스트와 레이블 입력데이터를 출력 output에 넣음
                #시간 딜레이를 주자
                #time.sleep(1)
                _, logits = outputs #outputs를 로짓에 넣음 이것을 softmax에 넣으면 0~1 사이로 결과가 출력됨
                
                pred = torch.argmax(F.softmax(logits), dim=1) #드디어 예측한다. argmax는 리스트(계산된 값)에서 가장 큰 값을 추출하여 pred에 넣는다. 0 ~1 사이의 값이 나올거임
                print('pred :', pred)
                # correct = pred.eq(labels) 
                showing__ = pred.eq(1) # 예측한 결과가 1과 같으면 showing이다   >> TRUE   SHOWING을 추출하려면 이것만 카운드하면 된다. 
                telling__ = pred.eq(0) # 예측한 결과가 0과 같으면 telling이다   >> FALSE
                
                #print('showing : ', showing__)
                #print('telling : ', telling__)
                
                
                showing_conunter.append(text)        
                #pred_ = round(float(pred))
                showing_conunter.append(pred)



            return showing_conunter 


        st_re = showtell_classfy(str(input_sent_chr)) # 캐릭터거 포함된 문장(전처리 완료된) 입력

        df = DataFrame(st_re)
        df_ = df[0::2] # 글만 추출
        df_label = df[1::2] # 레이블만 추출

        df_.reset_index(drop=True, inplace=True) #데이터를 합치기 위해서 초기화
        df_label.reset_index(drop=True, inplace=True)

        df_result = pd.concat([df_,df_label],axis=1) #합치기

        df_result.columns = ['sentence','show/tell']

        df_fin = df_result['show/tell'].value_counts(normalize=True)
        list(df_fin)
        showing_sentence_with_char = max(round(df_fin*100))

        # print("===============================================================")
        # print ('Character Descriptiveness : ', showing_sentence_with_char)
        # print("===============================================================")

        return showing_sentence_with_char



    ################################################
    #############  Emphasis on YOU  ################
    ################################################
    def EmphasisOnYou(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'I', 'my', 'me', 'mine', 'one']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
#         for i in filtered_chr_text__:
#             ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count


    # EmphasisOnYou_ = EmphasisOnYou(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnYou_
    # print ('=============================================')
    # print ('Emphasis on You :', EmphasisOnYou_)
    # print ('=============================================')



    #########################################
    ######### Emphasis on others  ###########
    #########################################
    def EmphasisOnOthers(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin',
                        'daughter', 'duplicate', 'ex', 'forefather', 'founder', 'gemini',
                        'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium', 'husband',
                        'in', 'law', 'match', 'mother', 'nephew', 'niece', 'parent', 'person', 'rear',
                        'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow',
                        'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
#         for i in filtered_chr_text__:
#             ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
    
    
        return char_total_count


    # EmphasisOnOthers_ = EmphasisOnOthers(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnOthers_
    # print ('=============================================')
    # print ('Emphasis on Others :', EmphasisOnOthers_)
    # print ('=============================================')


    character_descriptiveness = character_descrip(text)
    print("===============================================================")
    print ('Character Descriptiveness : ' , character_descriptiveness)
    print("===============================================================")


    number_of_characters = NumberofCharacters(text) 
    print ('=============================================')
    print ('Number of Characters :' , number_of_characters)
    print ('=============================================')


    EmphasisOnYou_ = EmphasisOnYou(text)
    print ('=============================================')
    print ('Emphasis on You :' , EmphasisOnYou_)
    print ('=============================================')


    EmphasisOnOthers_ = EmphasisOnOthers(text) 
    print ('=============================================')
    print ('Emphasis on Others :' , EmphasisOnOthers_)
    print ('=============================================')


    return character_descriptiveness, number_of_characters, EmphasisOnYou_, EmphasisOnOthers_



## 1000명의 개별 에세이 값을 각각 불러와서, 데이터프레임으로 결과값을 저장하고, 표준편차 계산하여 적용

In [4]:
#!/usr/bin/python
# -*- coding:utf-8 -*-

import json
import pandas as pd
import sys
import os

In [5]:
pwd

'/Users/kimkwangil/Documents/008_AI/Story_Analysis-master 8/CHARACTER_'

In [6]:
path = "/Users/kimkwangil/Documents/008_AI/Story_Analysis-master 8/CHARACTER_/ps_data/"
file_list = os.listdir(path)
file_list

['each_personal_essay_538.json',
 'each_personal_essay_492.json',
 'each_personal_essay_168.json',
 'each_personal_essay_884.json',
 'each_personal_essay_187.json',
 'each_personal_essay_50.json',
 'each_personal_essay_304.json',
 'each_personal_essay_754.json',
 'each_personal_essay_241.json',
 'each_personal_essay_611.json',
 'each_personal_essay_580.json',
 'each_personal_essay_979.json',
 'each_personal_essay_703.json',
 'each_personal_essay_353.json',
 'each_personal_essay_646.json',
 'each_personal_essay_216.json',
 'each_personal_essay_579.json',
 'each_personal_essay_129.json',
 'each_personal_essay_980.json',
 'each_personal_essay_596.json',
 'each_personal_essay_11.json',
 'each_personal_essay_345.json',
 'each_personal_essay_715.json',
 'each_personal_essay_200.json',
 'each_personal_essay_650.json',
 'each_personal_essay_484.json',
 'each_personal_essay_191.json',
 'each_personal_essay_938.json',
 'each_personal_essay_892.json',
 'each_personal_essay_742.json',
 'each_perso

In [7]:
from tqdm import tqdm

In [ ]:
essay_results = []

for file_name_raw in tqdm(file_list): # 파일이름을 하나씩 불러와서
    file_name = path  + file_name_raw #경로를 추가하고
    with open(file_name, 'r') as json_file : #JSON파일로 파일을 열어서
        ps_input_each_data = json.load(json_file) # JSON파일로 로드한다.
        input_data = json.dumps(ps_input_each_data)
        re_each_ess = character_all_section(input_data)
        essay_results.append(re_each_ess)

  0%|          | 0/981 [00:00<?, ?it/s]

model loadling~
pred_loader: <torch.utils.data.dataloader.DataLoader object at 0x7fae09af3490>
check!
text: ("In our Singaporean version of a Toyota minivan, we passed it almost daily.As a whole, my childhood was relatively sheltered, and the biggest problems I faced were losing my goggles or failing to remember how to brake on rollerskates- but that doesn't mean that I don't care, that I don't want to learn.",)


/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:210: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pred : tensor([1])
text: ("Few questions asked, and definitely none answered, land was considered a precious resource and no amount of history or character could prevent the destruction.Prompt: Person in history you would want to meetThe grotesque nature of Chicago's massive slaughterhouse and poverty-stricken, soot-infested neighborhoods earned it the nickname of 'The Black City' in the late 1800s, but despite the striking lack of sanitation, upon reading Erik Larson's The Devil in the White City, I would love the opportunity to see the dichotomy between the city's prosperous World's Fair in 1893 and the desolate alleys that coined the phrase.",)
pred : tensor([1])
text: ('I still have a list of possible ideas and introductions saved on my computer, and trust me, most would make for some fun reading material.',)
pred : tensor([1])
text: ('I might not have the resources or education to face these global problems right now, but as clich\\u00e9 as it sounds, I like to think that my futur

In [ ]:
essay_results

In [ ]:
# ===============================================================
# Character Descriptiveness :  33.0
# ===============================================================
# ['their', 'they', 'me', 'you', 'her', 'it', 'myself', 'i', 'my', 'them']
# ai_character_section.py:92: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Number of Characters : 92
# =============================================
# ['i', 'my', 'me']
# ai_character_section.py:338: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Emphasis on You : 60
# =============================================
# ['their', 'they', 'you', 'her', 'it', 'myself', 'them']
# ai_character_section.py:409: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
#   ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
# =============================================
# Emphasis on Others : 32
# =============================================
# (33.0, 92, 60, 32)

In [ ]:
df_r = pd.DataFrame(essay_results, columns =['char_desc', 'num_chars', 'emph_you', 'emph_others'])  #e데이터프레임에 넣기
df_r

In [ ]:
#1000명 데이터의 각 값의 평균 계산
char_desc_mean = df_r['char_desc'].mean()
char_desc_mean

In [ ]:
num_chars_mean = df_r['num_chars'].mean()
num_chars_mean

In [ ]:
emph_you_mean = df_r['emph_you'].mean()
emph_you_mean 

In [ ]:
emph_others_mean = df_r['emph_others'].mean()
emph_others_mean

In [ ]:
# 30명의 최종 평균값 계산
char_re_30 = [char_desc_mean, num_chars_mean, emph_you_mean, emph_others_mean]
char_re_30

## 1명의 입력데이터

In [ ]:
input_text = """Bloomington Normal is almost laughably cliché for a midwestern city. Vast swathes of corn envelop winding roads and the heady smell of BBQ smoke pervades the countryside every summer. Yet, underlying the trite norms of Normal is the prescriptive force of tradition—the expectation to fulfill my role as a female Filipino by playing Debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.So when I discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, I was fascinated by the music's novelty. Jazz guitar was not only evocative and creative, but also strangely liberating. I began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. And, although I did not know what would be the 'best' route for me to follow as a musician, the freedom to forge whatever path I felt was right seemed to be exactly what I needed; there were no expectations for me to continue in any particular way—only the way that suited my own desires.While journeying this trail, I found myself at Interlochen Arts Camp the summer before my junior year. Never before had I been immersed in an environment so conducive to musical growth: I was surrounded by people intensely passionate about pursuing all kinds of art with no regard for ideas of what art 'should' be. I knew immediately that this would be a perfect opportunity to cultivate my sound, unbounded by the limits of confining tradition. On the first day of camp, I found that my peer guitarist in big band was another Filipino girl from Illinois. Until that moment, my endeavors in jazz guitar had been a solitary effort; I had no one with whom to collaborate and no one against whom I could compare myself, much less someone from a background mirroring my own. I was eager to play with her, but while I quickly recognized a slew of differences between us—different heights, guitars, and even playing styles—others seemed to have trouble making that distinction during performances. Some even went as far as calling me 'other-Francesca.' Thus, amidst the glittering lakes and musky pine needles of Interlochen, I once again confronted Bloomington's frustrating expectations.After being mistaken for her several times, I could not help but view Francesca as a standard of what the 'female Filipino jazz guitarist' should embody. Her improvisatory language, comping style and even personal qualities loomed above me as something I had to live up to. Nevertheless, as Francesca and I continued to play together, it was not long before we connected through our creative pursuit. In time, I learned to draw inspiration from her instead of feeling pressured to follow whatever precedent I thought she set. I found that I grew because of, rather than in spite of, her presence; I could find solace in our similarities and even a sense of comfort in an unfamiliar environment without being trapped by expectation. Though the pressure to conform was still present—and will likely remain present in my life no matter what genre I'm playing or what pursuits I engage in—I learned to eschew its corrosive influence and enjoy the rewards that it brings. While my encounter with Francesca at first sparked a feeling of pressure to conform in a setting where I never thought I would feel its presence, it also carried the warmth of finding someone with whom I could connect. Like the admittedly trite conditions of my hometown, the resemblances between us provided comfort to me through their familiarity. I ultimately found that I can embrace this warmth while still rejecting the pressure to succumb to expectations, and that, in the careful balance between these elements, I can grow in a way that feels both like discove"""

In [ ]:
ps_char_re = character_all_section(input_text)
ps_char_re 

In [ ]:
type(ps_char_re )

In [ ]:
one_ps_char_desc = ps_char_re[0] #튜플에서 첫번재 인댁스 값 가져오기 : Character Descriptiveness

# 결과도출 1명의 Character Descriptiveness 계산결과
one_ps_char_desc

In [ ]:
ideal_mean = char_desc_mean #1000명 데이터의 각 값의 평균 값 전달

In [ ]:
min_ = int(ideal_mean-ideal_mean*0.6)
print('min_', min_)
max_ = int(ideal_mean+ideal_mean*0.6)
print('max_: ', max_)
div_ = int(((ideal_mean+ideal_mean*0.6)-(ideal_mean-ideal_mean*0.6))/3)
print('div_:', div_)


In [ ]:
cal_abs = abs(ideal_mean - one_ps_char_desc) # 개인 - 단체 값의 절대값계산

print('cal_abs 절대값 :', cal_abs)
compare = (one_ps_char_desc + ideal_mean)/7
print('compare :', compare)

if one_ps_char_desc > ideal_mean: # 개인점수가 평균보다 클 경우는 overboard
    if cal_abs > compare: # 개인점수가 개인평균차의 절대값보다 클 경우, 즉 차이가 많이 날경우
        print("Overboard")
    else: #차이가 많이 안나면
        print("Ideal")
        
    
elif one_ps_char_desc < ideal_mean: # 개인점수가 평균보다 작을 경우 lacking
    if cal_abs > compare: #차이가 많이나면 # 개인점수가  평균보다 작을 경우 Lacking이고 
        print("Lacking")
    else: #차이가 많이 안나면
        print ("Ideal")
        
else:
    print("Ideal")

    
    

## 1000명 에세이 데이터 처리 결과 저장

In [ ]:
import pickle
# save
with open('1000_essay_results.pickle', 'wb') as f:
    pickle.dump(essay_results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load
with open('data_ai_theme.pickle', 'rb') as f:
    result_most_simWords = pickle.load(f)